# Stock Trading

The cell below defines the **abstract class** whose API you need to implement. **Do NOT modify it** - use the dedicated cell further below for your implementation instead.

In [ ]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  
      

# abstract class to represent a stock trading platform
class AbstractStockTradingPlatform(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # adds transactionRecord to the set of completed transactions
    @abstractmethod
    def logTransaction(self, transactionRecord):
        pass

    # returns a list with all transactions of a given stockName,
    # sorted by increasing trade value. 
    # stockName : str
    @abstractmethod
    def sortedTransactions(self, stockName): 
        sortedList = []
        return sortedList    
    
    # returns a list of transactions of a given stockName with minimum trade value
    # stockName : str
    @abstractmethod
    def minTransactions(self, stockName): 
        minList = []
        return minList    
    
    # returns a list of transactions of a given stockName with maximum trade value
    # stockName : str
    @abstractmethod
    def maxTransactions(self, stockName): 
        maxList = []
        return maxList    

    # returns a list of transactions of a given stockName, 
    # with the largest trade value below a given thresholdValue.  
    # stockName : str
    # thresholdValue : double
    @abstractmethod
    def floorTransactions(self, stockName, thresholdValue): 
        floorList = []
        return floorList    

    # returns a list of transactions of a given stockName, 
    # with the smallest trade value above a given thresholdValue.  
    # stockName : str
    # thresholdValue : double
    @abstractmethod
    def ceilingTransactions(self, stockName, thresholdValue): 
        ceilingList = []
        return ceilingList    

        
    # returns a list of transactions of a given stockName,  
    # whose trade value is within the range [fromValue, toValue].
    # stockName : str
    # fromValue : double
    # toValue : double
    @abstractmethod
    def rangeTransactions(self, stockName, fromValue, toValue): 
        rangeList = []
        return rangeList    

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [ ]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE


# This class models a transaction record
class Trade:
    # Parameterized with all the relevant information associated to a single transaction record
    def __init__(self, name: str, price: float, quantity: int, time) -> None:
        self.name = name
        self.price = price
        self.quantity = quantity
        self.time = time

    # This helper method makes it easy to retrieve the trade value of a Trade object
    def get_trade_val(self) -> float:
        return self.price * self.quantity

    # Converting to a list makes Trade objects easier to work with in some applications
    def to_list(self) -> list:
        return [self.name, self.price, self.quantity, self.time]


# This class models the nodes used in the TradeTree class
class TradeNode:
    # Constants that represent node color
    RED = True
    BLACK = False

    def __init__(self, trade: Trade) -> None:
        # The trade value is interpreted as the key of a TradeNode object
        self.trade_val = trade.get_trade_val()

        # List of all Trade objects with the same trade value
        self.trades = [trade]

        # References to children nodes
        self.left = None
        self.right = None

        # The color associated to each instance of TradeNode
        self.color = TradeNode.RED


# This class models all information for a single stock name
# All transactions on a given stock will be stored here
# This has no model of any other stock names
# It implements a balanced search tree ADT using a left-leaning red-black binary search tree
# Each node in the tree is a TradeNode object
class TradeTree:
    # Initialize stock name and root node
    def __init__(self, stock_name: str) -> None:
        self.stock_name = stock_name
        self.root = None

    def put_trade(self, trade: Trade) -> None:
        # Ensure that the Trade object to be inserted matches the stock name of the current TradeTree object
        if trade.name != self.stock_name:
            raise ValueError("Invalid Stock Name")

        # Reassign root with updated TradeNode object
        self.root = self.__insert(trade, self.root)

        # Maintain invariant of coloring root node black
        self.root.color = TradeNode.BLACK

    def __insert(self, trade: Trade, node: TradeNode) -> TradeNode:
        # Recursive base case which inserts a new TradeNode object
        if node is None:
            return TradeNode(trade)

        # Use trade value as the key for inserting TradeNode objects into the TradeTree
        trade_val = trade.get_trade_val()

        # Recursive step case to determine appropriate insertion position
        if trade_val == node.trade_val:
            node.trades.append(trade)
        elif trade_val < node.trade_val:
            node.left = self.__insert(trade, node.left)
        elif trade_val > node.trade_val:
            node.right = self.__insert(trade, node.right)

        # Balance the TradeTree to maintain logarithmic height
        return TradeTree.__balance(node)

    def get_all_trades(self, node: TradeNode = None) -> list:
        # Base case where the root of the TradeTree has not been initialized
        if self.root is None:
            return []

        # Optional node parameter used to traverse some subtree
        if node is None:
            node = self.root

        # In order traversal of nodes
        all_trades = []
        if node.left is not None:
            all_trades = self.get_all_trades(node.left)
        all_trades.extend(node.trades)
        if node.right is not None:
            all_trades.extend(self.get_all_trades(node.right))

        return all_trades

    def get_min_trades(self) -> list:
        if self.root is None:
            return []

        # Iteratively traverse left until the bottom of the tree
        node = self.root
        while node.left is not None:
            node = node.left

        return node.trades

    def get_max_trades(self) -> list:
        if self.root is None:
            return []

        # Iteratively traverse right until the bottom of the tree
        node = self.root
        while node.right is not None:
            node = node.right

        return node.trades

    def get_floor_trades(self, high: float) -> list:
        node = self.root
        floor_trades = []

        while node is not None:
            # If trade value equal to threshold is found
            if node.trade_val == high:
                # Return transactions
                return node.trades

            # Else if trade value less than threshold
            elif node.trade_val < high:
                # Store transactions and go right
                floor_trades = node.trades
                node = node.right

            # Else if trade value greater than threshold
            elif node.trade_val > high:
                # Go left
                node = node.left

        return floor_trades

    def get_ceil_trades(self, low: float) -> list:
        node = self.root
        ceil_trades = []

        while node is not None:
            # If trade value equal to threshold is found
            if node.trade_val == low:
                # Return transactions
                return node.trades

            # Else if trade value greater than threshold
            elif node.trade_val > low:
                # Store transactions and go left
                ceil_trades = node.trades
                node = node.left

            # Else if trade value less than threshold
            elif node.trade_val < low:
                # Go right
                node = node.right

        return ceil_trades

    def get_trades_in_range(self, low: float, high: float, node: TradeNode = None) -> list:
        # Ensure that the low and high parameters are valid
        if low > high or low < 0:
            raise ValueError("Invalid Range")

        if self.root is None:
            return []

        if node is None:
            node = self.root

        trades_in_range = []

        # In order traversal if trade value is in range
        if low <= node.trade_val <= high:
            if node.left is not None:
                trades_in_range = self.get_trades_in_range(low, high, node.left)
            trades_in_range.extend(node.trades)
            if node.right is not None:
                trades_in_range.extend(self.get_trades_in_range(low, high, node.right))

        # Go right if trade value is less than lower bound
        elif node.trade_val < low and node.right is not None:
            trades_in_range = self.get_trades_in_range(low, high, node.right)

        # Go left if trade value is greater than higher bound
        elif node.trade_val > high and node.left is not None:
            trades_in_range = self.get_trades_in_range(low, high, node.left)

        return trades_in_range

    @staticmethod
    def __rotate_left(node: TradeNode) -> TradeNode:
        x = node.right
        node.right = x.left
        x.left = node
        x.color = node.color
        node.color = TradeNode.RED
        return x

    @staticmethod
    def __rotate_right(node: TradeNode) -> TradeNode:
        x = node.left
        node.left = x.right
        x.right = node
        x.color = node.color
        node.color = TradeNode.RED
        return x

    @staticmethod
    def __flip_colors(node: TradeNode) -> None:
        node.color = TradeNode.RED
        node.left.color = TradeNode.BLACK
        node.right.color = TradeNode.BLACK

    @staticmethod
    def __is_red(node: TradeNode) -> bool:
        return node.color == TradeNode.RED if node is not None else False

    # Maintains the structural invariants of a left-leaning red-black binary search tree
    @staticmethod
    def __balance(node: TradeNode) -> TradeNode:
        # Perform rotations if required
        if TradeTree.__is_red(node.right) and not TradeTree.__is_red(node.left):
            node = TradeTree.__rotate_left(node)
        if TradeTree.__is_red(node.left) and TradeTree.__is_red(node.left.left):
            node = TradeTree.__rotate_right(node)

        # Flip colors if required
        if TradeTree.__is_red(node.left) and TradeTree.__is_red(node.right):
            TradeTree.__flip_colors(node)

        return node

Use the cell below to implement the requested API. 

In [ ]:
# IMPLEMENT HERE THE REQUESTED API


# This class implements a hash table ADT using a Python's built-in dictionary data structure
# The keys are the stock names and the values are references to TradeTree objects
# Each method in this class does thorough error checking before calling operations on the TradeTree objects
class StockTradingPlatform(AbstractStockTradingPlatform):
    def __init__(self) -> None:
        self.STOCKS = ["Barclays", "HSBA", "Lloyds Banking Group", "NatWest Group", "Standard Chartered", "3i",
                       "Abrdn", "Hargreaves Lansdown", "London Stock Exchange Group", "Pershing Square Holdings",
                       "Schroders", "St. James's Place plc."]

        self.__trade_trees = {}

        for stock in self.STOCKS:
            self.__trade_trees[stock] = TradeTree(stock)

    def logTransaction(self, transactionRecord: list) -> None:
        trade = Trade(*transactionRecord)
        self.__validate_trade(trade)
        self.__trade_trees[trade.name].put_trade(trade)

    def sortedTransactions(self, stockName: str) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("sortedTransactions: Invalid Stock Name: " + stockName)

        return self.__trade_trees[stockName].get_all_trades()

    def minTransactions(self, stockName: str) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("minTransactions: Invalid Stock Name: " + stockName)

        return self.__trade_trees[stockName].get_min_trades()

    def maxTransactions(self, stockName: str) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("maxTransactions: Invalid Stock Name: " + stockName)

        return self.__trade_trees[stockName].get_max_trades()

    def floorTransactions(self, stockName: str, thresholdValue: float) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("floorTransactions: Invalid Stock Name: " + stockName)

        if thresholdValue < 0:
            raise ValueError("floorTransactions: Invalid Transaction Value: " + str(thresholdValue))

        return self.__trade_trees[stockName].get_floor_trades(thresholdValue)

    def ceilingTransactions(self, stockName: str, thresholdValue: float) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("ceilingTransactions: Invalid Stock Name: " + stockName)

        if thresholdValue < 0:
            raise ValueError("ceilingTransactions: Invalid Transaction Value: " + str(thresholdValue))

        return self.__trade_trees[stockName].get_ceil_trades(thresholdValue)

    def rangeTransactions(self, stockName: str, fromValue: float, toValue: float) -> list:
        if stockName not in self.STOCKS:
            raise ValueError("rangeTransactions: Invalid Stock Name: " + stockName)

        if fromValue > toValue or fromValue < 0 or toValue < 0:
            raise ValueError(
                "rangeTransactions: Invalid Range Bounds: fromValue: " + str(fromValue) + " toValue: " + str(toValue)
            )

        return self.__trade_trees[stockName].get_trades_in_range(fromValue, toValue)

    # Ensures that the transaction records to be inserted are valid
    def __validate_trade(self, trade: Trade) -> None:
        if trade.name not in self.STOCKS:
            raise ValueError("Invalid Stock Name: " + trade.name)

        if trade.quantity < 1:
            raise ValueError("Invalid Stock Quantity: " + str(trade.quantity))

        if trade.price <= 0.0:
            raise ValueError("Invalid Stock Price: " + str(trade.price))

The cell below provides helper code that you can use within your experimental framework to generate random transaction data. **Do NOT modify it**.

In [ ]:
# DO NOT MODIFY THIS CELL

import random
from datetime import timedelta
from datetime import datetime

class TransactionDataGenerator:
    def __init__(self):
        self.stockNames = ["Barclays", "HSBA", "Lloyds Banking Group", "NatWest Group", 
                      "Standard Chartered", "3i", "Abrdn", "Hargreaves Lansdown", 
                      "London Stock Exchange Group", "Pershing Square Holdings", 
                      "Schroders", "St. James's Place plc."]
        self.minTradeValue = 500.00
        self.maxTradeValue = 100000.00
        self.startDate = datetime.strptime('1/1/2022 1:00:00', '%d/%m/%Y %H:%M:%S')
        random.seed(20221603)
          
    # returns the name of a traded stock at random
    def getStockName(self):
        return random.choice(self.stockNames)

    # returns the trade value of a transaction at random
    def getTradeValue(self):
        return round(random.uniform(self.minTradeValue, self.maxTradeValue), 2)
    
    # returns a list of N randomly generated transactions,
    # where each transaction is represented as a list [stock name, price, quantity, timestamp]
    # N : int
    def generateTransactionData(self, N):   
        listTransactions = [[]]*N
        listDates = [self.startDate + timedelta(seconds=3*x) for x in range(0, N)]
        listDatesFormatted = [x.strftime('%d/%m/%Y %H:%M:%S') for x in listDates]
        for i in range(N):
            stockName = random.choice(self.stockNames)
            price = round(random.uniform(50.00, 100.00), 2)
            quantity = random.randint(10,1000)
            listTransactions[i] = [stockName, price, quantity, listDatesFormatted[i]]   
        return listTransactions

Use the cell below for the python code needed to realise your **experimental framework** (i.e., to generate test data, to instante the `StockTrading` class, to thorouhgly experiment with its API functions, and to experimentally measure their performance). You may use the previously provided ``TransactionDataGenerator`` class to generate random transaction data.

In [ ]:
# ADD YOUR EXPERIMENTAL FRAMEWORK CODE HERE

import random
import timeit


# This class manages the process of creating and testing instances of the StockTradingPlatform class
class ExperimentalFramework:
    # Constants that represent different cases to be examined
    LOG_RANDOM = 0
    LOG_SORTED = 1
    SORTED = 2
    MIN = 3
    MAX = 4
    FLOOR_RANDOM = 5
    FLOOR_EXISTING = 6
    CEILING_RANDOM = 7
    CEILING_EXISTING = 8
    RANGE_RANDOM = 9
    RANGE_ALL = 10

    # Parameterized with the maximum number of transactions to be logged, the step size, and the number of trials
    def __init__(self, n_transactions: int, n_step: int, n_trials: int) -> None:
        self.__n_transactions = n_transactions
        self.__n_step = n_step
        self.__n_trials = n_trials
        self.__generator = TransactionDataGenerator()
        self.__times = {case: {i: [] for i in self.get_n_transactions_list()} for case in range(11)}

    @staticmethod
    def __trade_value(transaction: list) -> float:
        return transaction[1] * transaction[2]

    # Generates random transactions and reassigns the stock names to be uniform
    def __gen_transactions_same_stock(self, case: int) -> list:
        transactions = self.__generator.generateTransactionData(self.__n_transactions)
        stock_name = self.__generator.getStockName()

        # Assign a particular randomly chosen stock name to each transaction
        for i in range(len(transactions)):
            transactions[i][0] = stock_name

        # Sort the generated transactions by trade value in the appropriate case
        if case == ExperimentalFramework.LOG_SORTED:
            transactions.sort(key = ExperimentalFramework.__trade_value, reverse = True)

        return transactions

    # Tests the execution time of a general API operation func with arguments *args
    def __test_ordered_op(self, n_curr: int, func: callable, case: int, args: list) -> list:
        start = timeit.default_timer()
        trades = func(*args)
        end = timeit.default_timer()
        self.__times[case][n_curr].append(end - start)
        return trades

    # A sequence of calls to __test_ordered_op() that examine the performance of all ordered API operations
    def __test_all_ordered_ops(self, n_curr: int, platform: StockTradingPlatform, stock_name: str) -> None:
        # Testing the sorted transactions operation
        sorted_trades = self.__test_ordered_op(n_curr, platform.sortedTransactions, ExperimentalFramework.SORTED, [
            stock_name
        ])

        # Testing the min transactions operation
        self.__test_ordered_op(n_curr, platform.minTransactions, ExperimentalFramework.MIN, [
            stock_name
        ])

        # Testing the max transactions operation
        self.__test_ordered_op(n_curr, platform.maxTransactions, ExperimentalFramework.MAX, [
            stock_name
        ])

        # Testing the floor transactions operation
        self.__test_ordered_op(n_curr, platform.floorTransactions, ExperimentalFramework.FLOOR_RANDOM, [
            stock_name, self.__generator.getTradeValue()
        ])
        self.__test_ordered_op(n_curr, platform.floorTransactions, ExperimentalFramework.FLOOR_EXISTING, [
            stock_name, random.choice(sorted_trades).get_trade_val()
        ])

        # Testing the ceiling transactions operation
        self.__test_ordered_op(n_curr, platform.ceilingTransactions, ExperimentalFramework.CEILING_RANDOM, [
            stock_name, self.__generator.getTradeValue()
        ])
        self.__test_ordered_op(n_curr, platform.ceilingTransactions, ExperimentalFramework.CEILING_EXISTING, [
            stock_name, random.choice(sorted_trades).get_trade_val()
        ])

        # Testing the range transactions operation
        range_random_args = [stock_name]
        range_random_args.extend(sorted([self.__generator.getTradeValue(), self.__generator.getTradeValue()]))
        self.__test_ordered_op(
            n_curr, platform.rangeTransactions, ExperimentalFramework.RANGE_RANDOM, range_random_args
        )
        range_all_args = [stock_name]
        range_all_args.extend(sorted([self.__generator.minTradeValue, self.__generator.maxTradeValue]))
        self.__test_ordered_op(
            n_curr, platform.rangeTransactions, ExperimentalFramework.RANGE_ALL, range_all_args
        )

    # Driver method that handles the logging of transactions and when to record execution times
    def __test_log(self, case: int) -> None:
        # Loop for the number of trials
        for _ in range(self.__n_trials):
            transactions = self.__gen_transactions_same_stock(case)
            platform = StockTradingPlatform()

            # Iterate through generated transactions with a particular randomly chosen stock name
            for i, transaction in enumerate(transactions):
                start = timeit.default_timer()
                platform.logTransaction(transaction)
                end = timeit.default_timer()

                # For desired multiples of step size
                if (i + 1) % self.__n_step == 0:
                    # Record log time
                    self.__times[case][i + 1].append(end - start)

                    # Examine performance of ordered API operations
                    if case == ExperimentalFramework.LOG_RANDOM:
                        self.__test_all_ordered_ops(i + 1, platform, transaction[0])

    # Calls the driver method __test_log() twice to perform all the required tests
    def run_tests(self) -> None:
        self.__test_log(ExperimentalFramework.LOG_RANDOM)
        self.__test_log(ExperimentalFramework.LOG_SORTED)

    # Displays the recorded execution times for each examined case
    def output_times(self) -> None:
        print("CASE #i = {num_trades_1: [trial_1, ..., trial_n], ..., num_trades_n: [trial_1, ..., trial_n]}\n")
        [print(f"CASE #{case} = {self.__times[case]}\n") for case in range(11)]

    # Returns a list of all multiples of step size less than or equal to the maximum number of transactions
    def get_n_transactions_list(self) -> list:
        return list(range(self.__n_step, self.__n_transactions + 1, self.__n_step))

    # Returns the execution times for a specified case
    def get_times(self, case: int) -> list:
        return [sum(time) / self.__n_trials for time in self.__times[case].values()]


if __name__ == '__main__':
    ef = ExperimentalFramework(10000, 100, 20)
    ef.run_tests()
    ef.output_times()

The cell below exemplifies **debug** code I will invoke on your submission - it does not represent an experimental framework (which should me much more comprehensive). **Do NOT modify it**. 

In [ ]:
# DO NOT MODIFY THIS CELL

import timeit

testPlatform = StockTradingPlatform()
testDataGen = TransactionDataGenerator()

numTransactions = 1000000
testData = testDataGen.generateTransactionData(numTransactions)

numRuns = 100

print("Examples of transactions:", testData[0], testData[numTransactions//2], testData[numTransactions-1])

#
# testing the logTransaction() API 
#
starttime = timeit.default_timer()
for i in range(numTransactions):
    testPlatform.logTransaction(testData[i])
endtime = timeit.default_timer()
print("\nExecution time to load", numTransactions, "transactions:", round(endtime-starttime,4))

#
# testing the various API functions
#
starttime = timeit.default_timer()
for i in range(numRuns):
    output = testPlatform.sortedTransactions(testDataGen.getStockName())
endtime = timeit.default_timer()
print("\nMean execution time sortedTransactions:", round((endtime-starttime)/numRuns,4))

starttime = timeit.default_timer()
for i in range(numRuns):
    output = testPlatform.minTransactions(testDataGen.getStockName())
endtime = timeit.default_timer()
print("\nMean execution time minTransactions:", round((endtime-starttime)/numRuns,4))

starttime = timeit.default_timer()
for i in range(numRuns):
    output = testPlatform.maxTransactions(testDataGen.getStockName())
endtime = timeit.default_timer()
print("\nMean execution time maxTransactions:", round((endtime-starttime)/numRuns,4))


starttime = timeit.default_timer()
for i in range(numRuns):
    output = testPlatform.floorTransactions(testDataGen.getStockName(), testDataGen.getTradeValue())
endtime = timeit.default_timer()
print("\nMean execution time floorTransactions:", round((endtime-starttime)/numRuns,4))


starttime = timeit.default_timer()
for i in range(numRuns):
    output = testPlatform.ceilingTransactions(testDataGen.getStockName(), testDataGen.getTradeValue())
endtime = timeit.default_timer()
print("\nMean execution time ceilingTransactions:", round((endtime-starttime)/numRuns,4))


starttime = timeit.default_timer()
for i in range(numRuns):
    rangeValues = sorted([testDataGen.getTradeValue(), testDataGen.getTradeValue()])
    output = testPlatform.rangeTransactions(testDataGen.getStockName(), rangeValues[0], rangeValues[1])
endtime = timeit.default_timer()
print("\nMean execution time rangeTransactions:", round((endtime-starttime)/numRuns,4))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random


class GraphPlotter:
    def __init__(self, ef: ExperimentalFramework) -> None:
        self.__ef = ef
        random.seed("Algorithms (COMP0005)")
        self.__ef.run_tests()
        self.__x = self.__ef.get_n_transactions_list()

    @staticmethod
    def __get_y_lim(ys: list) -> list:
        return [
            min([sorted(y)[0] * 0.75 for y in ys]),
            max([sorted(y)[int((len(y) - 1) * 0.97)] * 1.1 for y in ys])
        ]

    def __general_plot(self, op: str, ys: list, colors: list, funcs: list, labels: list = None) -> None:
        for i, (y, color, func) in enumerate(zip(ys, colors, funcs)):
            if labels:
                plt.plot(self.__x, y, color + ".", label = labels[i])
            else:
                plt.plot(self.__x, y, color + ".")

            plt.plot(self.__x, np.poly1d(np.polyfit(func(self.__x), y, 1))(func(self.__x)), color + "--")

        if labels:
            plt.legend(loc = "upper left")

        plt.title(op + " Time Vs. Transaction Count")
        plt.xlabel("Number of Transactions Under a Particular Stock")
        plt.ylabel("Execution Time (s)")
        plt.ylim(self.__get_y_lim(ys))
        plt.show()

    def plot_graphs(self) -> None:
        self.__general_plot(
            "Log Transaction",
            [
                self.__ef.get_times(ExperimentalFramework.LOG_RANDOM),
                self.__ef.get_times(ExperimentalFramework.LOG_SORTED)
            ],
            ["m", "b"],
            [np.log2, np.log2],
            ["Random Insertion Order", "Sorted Insertion Order"]
        )

        self.__general_plot(
            "Sorted Transactions",
            [self.__ef.get_times(ExperimentalFramework.SORTED)],
            ["b"],
            [lambda x: x]
        )

        self.__general_plot(
            "Min Transactions",
            [self.__ef.get_times(ExperimentalFramework.MIN)],
            ["r"],
            [np.log2]
        )

        self.__general_plot(
            "Max Transactions",
            [self.__ef.get_times(ExperimentalFramework.MAX)],
            ["g"],
            [np.log2]
        )

        self.__general_plot(
            "Floor Transactions",
            [
                self.__ef.get_times(ExperimentalFramework.FLOOR_RANDOM),
                self.__ef.get_times(ExperimentalFramework.FLOOR_EXISTING)
            ],
            ["r", "b"],
            [np.log2, np.log2],
            ["Random Threshold Values", "Threshold Values Existing in Tree"]
        )

        self.__general_plot(
            "Ceiling Transactions",
            [
                self.__ef.get_times(ExperimentalFramework.CEILING_RANDOM),
                self.__ef.get_times(ExperimentalFramework.CEILING_EXISTING)
            ],
            ["g", "b"],
            [np.log2, np.log2],
            ["Random Threshold Values", "Threshold Values Existing in Tree"]
        )

        self.__general_plot(
            "Range Transactions",
            [
                self.__ef.get_times(ExperimentalFramework.RANGE_RANDOM),
                self.__ef.get_times(ExperimentalFramework.RANGE_ALL)
            ],
            ["c", "b"],
            [lambda x: x, lambda x: x],
            ["Random Range", "Full Range"]
        )


if __name__ == '__main__':
    plotter = GraphPlotter(ExperimentalFramework(10000, 100, 20))
    plotter.plot_graphs()